In [1]:
#!/usr/bin/python
# encoding=utf8

import matplotlib.pyplot as plt
import numpy as np
from scipy import polyval, polyfit
from scipy.optimize import curve_fit

### external codes
from redTools import *
from Kcorrect import *
  
########################################################### Begin

inFile  = 'ESN_HI_catal.csv'   # input catalog
table   = np.genfromtxt(inFile , delimiter=',', filling_values=-1, names=True, dtype=None)

table = extinctionCorrect(table)
table = Kcorrection(table)

## purifying input data
index, = np.where(table['Sba']>0.01)
table = trim(table, index)

index, = np.where(table['Wba']>0.01)
table = trim(table, index)

index, = np.where(table['R50_g']>0)
table = trim(table, index)

index, = np.where(table['R50_r']>0)
table = trim(table, index)

index, = np.where(table['R50_i']>0)
table = trim(table, index)

index, = np.where(table['R50_z']>0)
table = trim(table, index)

index, = np.where(table['R50_w1']>0)
table = trim(table, index)

Sba = table['Sba']  # SDSS b/a
Wba = table['Wba']  # WISE b/a
Re_g = table['R50_g'] #half light radius for different bands
Re_r = table['R50_r']
Re_i = table['R50_i']
Re_z = table['R50_z']
Re_w1 = table['R50_w1']
Re_w2 = table['R50_w2']
Re_i_z = (1./2)*(Re_i + Re_z) #i+z half light radius

In [2]:
def halflight(reg, rer, rei, rez):
    m_g = 0.71 # slopes for halflight model
    m_r = 0.76
    m_i = 0.78
    m_z = 0.82

    b_g = 0.077 # y-intercepts for halflight model
    b_r = 0.072
    b_i = 0.069
    b_z = 0.068
    
    W = np.ones(shape = (4,len(reg)))  # group them into one array so we can remove them easier as we go
    W[0] = reg
    W[1] = rer
    W[2] = rei
    W[3] = rez
    prox = np.median(W.T,1)

    Z = np.argwhere(abs(W[3]-prox) < 0.2)    # returns the indicies where the difference is less than 0.2 
    print m_z*W[3, Z]+b_z, 'z'    # return halflight radius of for those indicies
    W = np.delete(W, Z, 1)         # remove the used indicies from the halflight list
    prox = np.delete(prox, Z)      # remove the used indicies from the proxy list
        
    I = np.argwhere(abs(W[2]-prox) < 0.2) # same thing for next band
    print m_i*W[2, I]+b_i, 'I'
    W = np.delete(W, I, 1)
    prox = np.delete(prox, I)
        
    R = np.argwhere(abs(W[1]-prox) < 0.2)
    print m_r*W[1, R]+b_r, 'r'
    W = np.delete(W, R, 1)
    prox = np.delete(prox, R)
        
    G = np.argwhere(abs(W[0]-prox) < 0.2)
    print m_g*W[0, G]+b_g, 'g'
    W = np.delete(W, G, 1)
    prox = np.delete(prox, G)
        
    if len(W[0,:]) == 0:
        print 'good'
    else:
        print 'not quite', W.shape

In [ ]:
def ba(Sba):
    m_sba = 0.852 # slope for Wba vs Sba
    b_sba = 0.1246 # y-intercept for Wba vs Sba
    print m_sba*Sba+b_sba, 'Wba'

In [3]:
halflight(Re_g, Re_r, Re_i, Re_z)